# 5 Modeling<a id='5_Modeling'></a>

## 5.1 Contents<a id='5.1_Contents'></a>
* [5 Modeling](#5_Modeling)
  * [5.1 Contents](#5.1_Contents)
  * [5.2 Introduction](#5.2_Introduction)
  * [5.3 Imports](#5.3_Imports)
  * [5.4 Load model](#5.4_Load_Model)
  * [5.5 Load data](#5.5_Load_Data)
  * [5.6 Refit model - 30056 features](#5.6_refit_model_30000)
      * [5.6.1 Performance of the model](#5.6.1_performance_model_30000)
  * [5.7 Refit model - best 100 features](#5.7_refit_100)
      * [5.7.1 Performance of the model](#5.7.1_performance_model_100)
  * [5.8 Refit model - best 50 features](#5.8_refit_50)
      * [5.8.1 Performance of the model](#5.8.1_performance_model_50)
  * [5.9 Conclusion](#5.9_conclusion)
  

## 5.2 Introduction<a id='5.2_Introduction'></a>

In this notebook, we use the best model that we built in the prevous step. We try out the model with 3 different numbers of features. In general, one should prefer a model with less number of features for the sake of simplicity and the computational efficiency. In addition to that, the performance of the model is also crucial for decision making. We determine the optimal number of features.  

## 5.3 Imports<a id='5.3_Imports'></a>

In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

from sklearn.model_selection import cross_validate
from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score,accuracy_score, mean_squared_error, mean_absolute_error, precision_recall_fscore_support
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import make_pipeline
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier 
from sklearn import metrics
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression, chi2
from sklearn.ensemble import ExtraTreesClassifier

from ipynb.fs.full.auxiliary_functions import save_file

%matplotlib inline


## 5.4 Load Model<a id='5.4_Load_Model'></a>

In [2]:
expected_model_version = '1.0'
model_path = '../Outputs/best_model.pkl'
if os.path.exists(model_path):
    with open(model_path, 'rb') as f:
        best_model = pickle.load(f)
    if best_model.version != expected_model_version:
        print("Expected model version doesn't match version loaded")
    if best_model.sklearn_version != sklearn_version:
        print("Warning: model created under different sklearn version")
else:
    print("Expected model not found")

## 5.5 Load Data<a id='5.5_Load_Data'></a>

In [3]:
df_final = pd.read_csv('df_final.csv')

In [4]:
df_final.head()

,member_id,icd_I10,icd_I10_chronic,icd_I739,icd_I739_chronic,num_of_visit,v1,v1_re_adm,v1_er_to_inp,v1_days_to_prev,...,ndc#_59746038410_refill,ndc#_68382013710_refill,ndc#_16252060102_refill,ndc#_10135018210_refill,ndc#_832102510_refill,ndc#_71093012105_refill,ndc#_43598075360_refill,ndc#_64380086106_refill,num_of_drugs,num_of_tests
0,nr+43zfHQPKSwx2IJuJI5Q==,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.0,311.0
1,Vwf7mI0tTGC4amYJPD6uJg==,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.0,132.0
2,TRudRczSQt6dwb6EeZ1RLA==,0.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,133.0,115.0
3,VjG87+cBSB2B1+loMmoHCg==,0.0,0.0,1.0,1.0,2.0,1.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202.0,283.0
4,7CfP6Hq5Qy6J0rXIuTc7kw==,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102.0,NaN


In [5]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1941 entries, 0 to 1940
Columns: 30057 entries, member_id to num_of_tests
dtypes: float64(30056), object(1)
memory usage: 445.1+ MB


Next we set `member_id` as index. 

In [6]:
df_final.set_index('member_id', inplace=True)
df_final.head()

,icd_I10,icd_I10_chronic,icd_I739,icd_I739_chronic,num_of_visit,v1,v1_re_adm,v1_er_to_inp,v1_days_to_prev,v1_len,...,ndc#_59746038410_refill,ndc#_68382013710_refill,ndc#_16252060102_refill,ndc#_10135018210_refill,ndc#_832102510_refill,ndc#_71093012105_refill,ndc#_43598075360_refill,ndc#_64380086106_refill,num_of_drugs,num_of_tests
member_id,,,,,,,,,,,,,,,,,,,,,
nr+43zfHQPKSwx2IJuJI5Q==,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.0,311.0
Vwf7mI0tTGC4amYJPD6uJg==,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.0,132.0
TRudRczSQt6dwb6EeZ1RLA==,0.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,133.0,115.0
VjG87+cBSB2B1+loMmoHCg==,0.0,0.0,1.0,1.0,2.0,1.0,0.0,0.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202.0,283.0
7CfP6Hq5Qy6J0rXIuTc7kw==,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102.0,NaN


We change the order of the columns in order to define X and y using iloc().

In [7]:
col=list(df_final.columns)
ordered_col=['icd_I10', 'icd_I739', 'icd_I10_chronic', 'icd_I739_chronic']+col[4:]
# reorder the columns of df_final

df_final=df_final[ordered_col]
df_final.head()

,icd_I10,icd_I739,icd_I10_chronic,icd_I739_chronic,num_of_visit,v1,v1_re_adm,v1_er_to_inp,v1_days_to_prev,v1_len,...,ndc#_59746038410_refill,ndc#_68382013710_refill,ndc#_16252060102_refill,ndc#_10135018210_refill,ndc#_832102510_refill,ndc#_71093012105_refill,ndc#_43598075360_refill,ndc#_64380086106_refill,num_of_drugs,num_of_tests
member_id,,,,,,,,,,,,,,,,,,,,,
nr+43zfHQPKSwx2IJuJI5Q==,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.0,311.0
Vwf7mI0tTGC4amYJPD6uJg==,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.0,132.0
TRudRczSQt6dwb6EeZ1RLA==,0.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,133.0,115.0
VjG87+cBSB2B1+loMmoHCg==,0.0,1.0,0.0,1.0,2.0,1.0,0.0,0.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202.0,283.0
7CfP6Hq5Qy6J0rXIuTc7kw==,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102.0,NaN


We change the dtype of the target features to make sure that our model will work properly.

In [8]:
df_final['icd_I10']=df_final['icd_I10'].astype('int')
df_final['icd_I739']=df_final['icd_I739'].astype('int')
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1941 entries, nr+43zfHQPKSwx2IJuJI5Q== to ZtdvAJtHT/uuwJWbulElSA==
Columns: 30056 entries, icd_I10 to num_of_tests
dtypes: float64(30054), int64(2)
memory usage: 445.1+ MB


In the following cell, we take care of abnormalities. We replace the missing values with 0. Furthermore, we make all the values positive using abs() function. 

In [9]:
df_final.replace([np.inf, -np.inf], np.nan, inplace=True)
df_final.replace([np.inf, -np.inf], np.nan, inplace=True)
df_final.fillna(0, inplace=True)

In [10]:
df_final=df_final[df_final.columns].abs()

## 5.6 Refit model - 30056 features <a id=5.6_refit_model_30000><a/> 

First we refit the best model that we built using all 30056 features. 

In [11]:
X=df_final.iloc[:, 2:]
y=df_final.iloc[:, 0:2]
print(X.shape, y.shape)

(1941, 30054) (1941, 2)


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

In [13]:
best_model.fit(X_train, y_train)

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(n_estimators=30, random_state=47))])

In [14]:
cv_results = cross_validate(best_model, X_train, y_train, scoring='accuracy', cv=5, n_jobs=-1)

In [15]:
cv_results['test_score']

array([0.85661765, 0.81617647, 0.83455882, 0.76752768, 0.80811808])

In [16]:
accuracy_mean, accuracy_std= np.round(np.mean(cv_results['test_score']), 4), np.round(np.std(cv_results['test_score']), 4)
accuracy_mean, accuracy_std

(0.8166, 0.0297)

In [17]:
y_pred_train=best_model.predict(X_train)
y_pred_test=best_model.predict(X_test)

In [18]:
print('ROC AUC:', roc_auc_score(y_test, y_pred_test))

ROC AUC: 0.8393857529314503


In [20]:
print(classification_report(y_test, y_pred_test, target_names=["I10", "I739"]))  

              precision    recall  f1-score   support

         I10       0.87      0.97      0.92       380
        I739       0.98      0.66      0.79       212

   micro avg       0.90      0.86      0.88       592
   macro avg       0.93      0.81      0.85       592
weighted avg       0.91      0.86      0.87       592
 samples avg       0.87      0.86      0.87       592



## 5.6.1 Performance of the model <a id=5.6.1_performance_model_30000><a/>      

We observe the following:

* Using the cross-validation we see that the average of the accuracy scores with 5-folds is 81% approximately. The standard deviation of the scores is 2.9% approximately. This indicates that the variance of the scores is significant. 

* The classification report shows that the patients with 'I10' were classified correctly at a rate of 97%. On the other hand, the patients with 'I739' were classified correctly at a lower rate 66% approximately. 


## 5.7 Refit model - best 100 features <a id=5.7_refit_100><a/>   

In [21]:
bestfeatures = SelectKBest(score_func=chi2, k=100)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['features','scores']  #naming the dataframe columns
featureScores.nlargest(100,'scores').head(30) #print 100 best features

,features,scores
20,v2_days_to_prev,2305.372136
6,v1_days_to_prev,1561.350298
34,v3_days_to_prev,1339.846479
1,icd_I739_chronic,1133.172308
48,v4_days_to_prev,658.510282
160,v12_days_to_prev,476.908073
30053,num_of_tests,417.215788
30052,num_of_drugs,416.337624
132,v10_days_to_prev,246.103004
146,v11_days_to_prev,204.106950


In [22]:
df_best_100_features=df_final[['icd_I10', 'icd_I739']+list(featureScores.nlargest(100, 'scores')['features'])]
df_best_100_features.head()

,icd_I10,icd_I739,v2_days_to_prev,v1_days_to_prev,v3_days_to_prev,icd_I739_chronic,v4_days_to_prev,v12_days_to_prev,num_of_tests,num_of_drugs,...,ndc#_17478071511_rate,ndc#_53885020801_rate,ndc#_70461041910_rate,ndc#_59676036001_rate,ndc#_60505279500_rate,ndc#_781620693_rate,ndc#_53885004601_rate,ndc#_23114501_rate,ndc#_169413212_rate,v15_len
member_id,,,,,,,,,,,,,,,,,,,,,
nr+43zfHQPKSwx2IJuJI5Q==,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,311.0,49.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Vwf7mI0tTGC4amYJPD6uJg==,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,132.0,41.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TRudRczSQt6dwb6EeZ1RLA==,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,115.0,133.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
VjG87+cBSB2B1+loMmoHCg==,0.0,1.0,267.0,0.0,0.0,1.0,0.0,0.0,283.0,202.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7CfP6Hq5Qy6J0rXIuTc7kw==,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
X=df_best_100_features.iloc[:, 2:]
y=df_best_100_features.iloc[:, 0:2]

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

In [25]:
best_model.fit(X_train, y_train)

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(n_estimators=30, random_state=47))])

In [26]:
cv_results = cross_validate(best_model, X_train, y_train, scoring='accuracy', cv=5, n_jobs=-1)

In [27]:
cv_results['test_score']

array([0.97426471, 0.97794118, 0.99264706, 0.9704797 , 0.97785978])

In [28]:
accuracy_mean, accuracy_std= np.round(np.mean(cv_results['test_score']), 4), np.round(np.std(cv_results['test_score']), 4)
accuracy_mean, accuracy_std

(0.9786, 0.0075)

In [29]:
y_pred_train=best_model.predict(X_train)
y_pred_test=best_model.predict(X_test)

In [30]:
print('ROC AUC:', roc_auc_score(y_test, y_pred_test)) 

ROC AUC: 0.9861930942659556


In [31]:
print(classification_report(y_test, y_pred_test, target_names=["I10", "I739"]))  

              precision    recall  f1-score   support

         I10       0.99      0.97      0.98       374
        I739       1.00      1.00      1.00       219

   micro avg       0.99      0.98      0.99       593
   macro avg       0.99      0.98      0.99       593
weighted avg       0.99      0.98      0.99       593
 samples avg       0.99      0.99      0.99       593



## 5.7.1 Performance of the model <a id=5.7.1_performance_model_100><a/>  

The results highlight the following:

* The average of the accuracy scores in the cross-validation with 5-folds is about 97%. This is much higher than the previous model. The standard deviation of the scores is 0.7% approximately which implies that the variance of the scores is fairly small. 

* The classification report tells us that the patients with 'I10' were classified correctly at a rate of 97%. The patients with 'I739' were classified correctly at a higher rate of approximately 100%. 


## 5.8 Refit model - best 50 features <a id=5.8_refit_50><a/>     

In [32]:
df_best_50_features=df_final[['icd_I10', 'icd_I739']+list(featureScores.nlargest(50, 'scores')['features'])]
df_best_50_features.head()

,icd_I10,icd_I739,v2_days_to_prev,v1_days_to_prev,v3_days_to_prev,icd_I739_chronic,v4_days_to_prev,v12_days_to_prev,num_of_tests,num_of_drugs,...,ndc#_69452015120_rate,v1_len,ndc#_65702072910_rate,ndc#_65862023703_rate,ndc#_70461020101_rate,ndc#_93360982_rate,ndc#_59310058020_rate,ndc#_58160081912_rate,ndc#_33332041710_rate,ndc#_49281040165_rate
member_id,,,,,,,,,,,,,,,,,,,,,
nr+43zfHQPKSwx2IJuJI5Q==,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,311.0,49.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Vwf7mI0tTGC4amYJPD6uJg==,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,132.0,41.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TRudRczSQt6dwb6EeZ1RLA==,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,115.0,133.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
VjG87+cBSB2B1+loMmoHCg==,0.0,1.0,267.0,0.0,0.0,1.0,0.0,0.0,283.0,202.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7CfP6Hq5Qy6J0rXIuTc7kw==,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
X=df_best_50_features.iloc[:, 2:]
y=df_best_50_features.iloc[:, 0:2]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

In [34]:
best_model.fit(X_train, y_train)

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(n_estimators=30, random_state=47))])

In [35]:
cv_results = cross_validate(best_model, X_train, y_train, scoring='accuracy', cv=5, n_jobs=-1)

In [36]:
cv_results['test_score']

array([0.97794118, 0.98529412, 0.98161765, 0.98154982, 0.98892989])

In [37]:
accuracy_mean, accuracy_std= np.round(np.mean(cv_results['test_score']), 4), np.round(np.std(cv_results['test_score']), 4)
accuracy_mean, accuracy_std

(0.9831, 0.0037)

In [38]:
y_pred_train=best_model.predict(X_train)
y_pred_test=best_model.predict(X_test)

In [46]:
print('ROC AUC:', roc_auc_score(y_test, y_pred_test))

ROC AUC: 0.994


In [41]:
print(classification_report(y_test, y_pred_test, target_names=["I10", "I739"]))  

              precision    recall  f1-score   support

         I10       1.00      0.98      0.99       375
        I739       1.00      1.00      1.00       218

   micro avg       1.00      0.98      0.99       593
   macro avg       1.00      0.99      0.99       593
weighted avg       1.00      0.98      0.99       593
 samples avg       1.00      0.99      0.99       593



## 5.8.1 Performance of the model <a id=5.8.1_performance_model_50><a/> 

The following are worth noting:

* The average of the accuracy scores in the cross-validation with 5-folds is about 98%. This is about 1% higher than the previous model's score. The standard deviation of the scores is approximately 0.3%, which is also less than the counterpart of the previous model.  

* The classification report shows that the patients with 'I10' were classified correctly at a rate of 98%. The patients with 'I739' were classified correctly at a higher rate of approximately 100%. 

* Overall the performance of this model is the best.


## 5.9 Conclusion <a id=5.9_conclusion><a/>

The model with only 50 features has a higher accuracy score in comparison to the first two models. Not only that, it has a lower variance of the scores which implies that the model depends less on the data points that have been chosen. Furthermore, this model has the highest f1 score as well as the highest recall for both classes. In general, one should prefer a model with less number of features for the sake of simplicity and the computational efficiency. Given the fact that our model is doing better with less number of features we choose our last model with only 50 features.